In [0]:
import random
import numpy as np

def is_primitive(alpha, q):
	n = []
	for i in range(1, q):
		a = (alpha**i) % q
		if a in n: return False
		n.append(a)
	return True

In [117]:
if __name__ == '__main__':
  prime_no = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]
  q = random.choice(prime_no)
  alpha = 2
  while not is_primitive(alpha, q):
    alpha += 1
  print(f"Q = {q} Alpha = {alpha}")
  clients = {'A': {}, 'B': {} }
  for k, v in clients.items():
    x = np.random.randint(2, q)
    new=x+q
    new1=new*q
    complex_1=((x+q)*q)%alpha
    c2=complex_1**2
    complex_1=c2+complex_1
    c2=complex_1**2
    new_complex_1=c2+complex_1
    y = (alpha**new_complex_1) % q
    y=y**3
    v['X'] = new_complex_1
    v['Y'] = y
  x_a = clients['A']['X']
  y_a = clients['A']['Y']
  x_b = clients['B']['X']
  y_b = clients['B']['Y']
  key_a = (y_b**x_a) % q
  key_b = (y_a**x_b) % q
  clients['A']['KEY'] = key_a
  clients['B']['KEY'] = key_b
  for k, v in clients.items():
    print(f"For client {k}")
    print(f"  X = {v['X']} Y = {v['Y']} Key = {v['KEY']}")



Q = 3 Alpha = 2
For client A
  X = 6 Y = 1 Key = 1
For client B
  X = 6 Y = 1 Key = 1


**Implementation combining own idea with crc**

In [116]:
import random
import numpy as np

def is_primitive(alpha, p):
	n = []
	for i in range(1, p):
		a = (alpha**i) % p
		if a in n: return False
		n.append(a)
	return True
   
def xor(a, b): 
    result = [] 
    for i in range(1, len(b)): 
        if a[i] == b[i]: 
            result.append('0') 
        else: 
            result.append('1') 
   
    return ''.join(result)
    
def mod2div(divident, divisor): 
    pick = len(divisor)  
    tmp = divident[0 : pick] 
   
    while pick < len(divident): 
   
        if tmp[0] == '1': 
            tmp = xor(divisor, tmp) + divident[pick] 
   
        else:    
            tmp = xor('0'*pick, tmp) + divident[pick] 
        pick += 1
    if tmp[0] == '1': 
        tmp = xor(divisor, tmp) 
    else: 
        tmp = xor('0'*pick, tmp) 
   
    checkword = tmp 
    return checkword 
    
def encodeData(data, key): 
   
    l_key = len(key) 
    appended_data = data + '0'*(l_key-1) 
    remainder = mod2div(appended_data, key) 
    codeword = data + remainder 
    return codeword
        

    
def decodeData(data, key): 
   
    l_key = len(key)  
    appended_data = data + '0'*(l_key-1) 
    remainder = mod2div(appended_data, key) 
   
    return remainder 
 

def DecimalToBinary(num):
    pol = ""
    while num > 0:
        j = num % 2
        num = num // 2
        pol = pol + str(j)
    pol = pol[::-1]    
    return pol
    
def binaryToDecimal(binary): 
    binary = int(binary)  
    binary1 = binary 
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return decimal
    
def Alice(poly):
    print('************** ALICE ***************')
    prime_no = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47] 
    p = random.choice(prime_no)
    q=p
    print('P : '+str(p))
    alpha = 2
    while not is_primitive(alpha, p):
        alpha=alpha + 1
    print('Alpha : '+str(alpha))
    a = np.random.randint(2, p)
    print('a : '+str(a))
    new=a+p
    new1=new*p
    complex_1=((a+p)*p)%alpha
    c2=complex_1**2
    complex_1=c2+complex_1
    c2=complex_1**2
    new_complex_1=c2+complex_1
    y = (alpha**new_complex_1) % p
    y=y**3
    A = y
    print('A : '+str(A))
    A = DecimalToBinary(A)
    print("Binary of A",A)
    crcOfA = encodeData(A,poly)
    print("CRC of A",crcOfA)
    print('*** Transfering control to Bob ***')
    crcOfB = Bob(p,alpha,crcOfA)
    print("CRC of B",crcOfB)
    print('*** Transfering control to Alice ***')
    remB = decodeData(crcOfB,poly)
    print("remB",remB)
    
    exp = '0'*(len(poly)-1)
    d=len(poly)-1
    if remB == exp:
        B = crcOfB[:-d]
        B = binaryToDecimal(B)
        print('B received by Alice : '+str(B))
        Ka = (B**new_complex_1) % p
        print('Secret Key Generated by Alice : '+ str(Ka))

def Bob(p,alpha,crcOfA):
    b = np.random.randint(2, p)
    print('b : '+str(b))
    new=b+p
    new1=new*p
    complex_1=((b+p)*p)%alpha
    c2=complex_1**2
    complex_1=c2+complex_1
    c2=complex_1**2
    new_complex_1=c2+complex_1
    y = (alpha**new_complex_1) % p
    y=y**3
    B = y
    print('B : '+str(B))
    B = DecimalToBinary(B)
    print("Binart of B",B)
    crcOfB = encodeData(B,poly)
    print("CRC of B",crcOfB)
    remA = decodeData(crcOfA,poly)
    print("remA of B",remA)
    d=len(poly)-1
    exp = '0'*d
    if remA == exp:
        A = crcOfA[:-d]
        A = binaryToDecimal(A)
        print('A received by Bob : '+str(A))
        Kb = (A**new_complex_1) % p
        print('Secret Key Generated by Bob : '+ str(Kb))
    return crcOfB

if __name__ == '__main__':
    global poly
    poly = random.randrange(2, 100,1)
    poly = DecimalToBinary(poly)
    print("Poly",poly)
    
    Alice(poly)

Poly 111101
************** ALICE ***************
P : 11
Alpha : 2
a : 10
A : 729
Binary of A 1011011001
CRC of A 101101100110110
*** Transfering control to Bob ***
b : 8
B : 729
Binart of B 1011011001
CRC of B 101101100110110
remA of B 00000
A received by Bob : 729
Secret Key Generated by Bob : 3
CRC of B 101101100110110
*** Transfering control to Alice ***
remB 00000
B received by Alice : 729
Secret Key Generated by Alice : 3
